Лабораторная работа 4. Яровенко Максим, ИУ5Ц-102Б. Вариант 22

# **Задание**

Вычислительный центр фирмы состоит из ***m*** главных серверов коллективного пользования. Число работающих в в центре программистов в любой момент времени равно ***k***. Каждый программист готовит свою программу и через терминал передает ее на сервер для выполнения, куда она сразу попадает. Время подготовки программ имеет экспоненциальное распределение со средним значением $t_1$ мин. Время выполнения программы на любом из серверов имеет экспоненциальное распределение со средним значением $t_2$ мин. Каждый программист ожидает ответа от сервера, прежде, чем начнет писать следующую программу.

Найти (теоретически и экспериментально):

- вероятность того, что программа не будет выполнена сразу же, как только она поступила на терминал;
- среднее время до получения пользователем результатов реализации;
- среднее количество программ, ожидающих выполнения на сервере.

Требуется выполнить расчет при заданном по варианту значении ***m***, а также при ***m=1***.

Задание по варианту:

In [ ]:
Variant<-22
set.seed(Variant)
k<-sample(c(10:25),1)
m<-sample(c(3:6),1)
t1<-sample(c(14:20),1)
t2<-sample(c(2:5),1)
View(data.frame(k,m,t1,t2))

k,m,t1,t2
<int>,<int>,<int>,<int>
15,3,15,3


# **Решение**

Для ***m*** по варианту

---



Экспериментальная часть:

In [ ]:
Tend <- 10000

tsys <- 0

progs <- data.frame()
for (i in c(1:k)) {
  progs <- rbind(progs, c(0, 0, 0))
}
colnames(progs) <- c("end", "start", "deploy_num")

for (i in 1:k) {
  progs$end[i] <- tsys + rexp(1, 1 / t1)
  progs$start[i] <- 0
  progs$deploy_num <- 0
}



servers <- data.frame()
for (i in c(1:m)) {
  servers <- rbind(servers, c(0, 0, 0))
}
colnames(servers) <- c("end", "start", "prog_num")

for (i in 1:m) {
  servers$end[i] <- Tend + 1
  servers$start[i] <- 0
  servers$prog_num <- 0
}

counter_deploy <- 0
counter_ready <- 0


system_count <- 1
last_switch <- 0

Pn_vec <- c(0,0)
Loch_vec <- c(0)

Tget_vec <- data.frame()
Tget_vec <- rbind(Tget_vec, c(0,0))

colnames(Tget_vec) <- c("time", "ready_flag")

queue <- c()

ready_time <- Tend + 1

#функция проверяет, есть ли свободные сервера
if_free_servers <- function() {
  for (i in 1:m) {
    if (servers$prog_num[i] == 0) {
      return (1)
    }
  }
  return (0)
}


while (tsys < Tend) {

  plus1 <- length(queue) + 1
  if (is.na(Loch_vec[plus1])) {Loch_vec[plus1] <- 0}
  Loch_vec[plus1] <- Loch_vec[plus1] + tsys - last_switch
  last_switch <- tsys

  for (i in 1:k) {
    if (tsys == progs$end[i]){

      if (counter_deploy - counter_ready < m) { #если есть свободные сервера
        Pn_vec[1] <- Pn_vec[1] + 1
      }
      else {Pn_vec[2] <- Pn_vec[2] + 1} #если сервера заняты

      counter_deploy <- counter_deploy + 1
      progs$deploy_num[i] <- counter_deploy
      Tget_vec <- rbind(Tget_vec, c(0,0))
      Tget_vec$time[counter_deploy] <- progs$end[i]
      Tget_vec$ready_flag[counter_deploy] <- 0

      progs$end[i] <- Tend + 1

      queue <- append(queue, counter_deploy)
    }
  }


  for (i in 1:m) {
    if (tsys == servers$end[i]) {
      counter_ready <- counter_ready + 1
      Tget_vec$time[counter_ready] <- tsys - Tget_vec$time[counter_ready]
      Tget_vec$ready_flag[counter_ready] <- 1

      for (j in 1:k) {
        if (progs$deploy_num[j] == servers$prog_num[i]) {
          progs$start[j] = tsys
          progs$end[j] = tsys + rexp(1, 1 / t1)
        }
      }

      servers$prog_num[i] <- 0
      servers$end[i] <- Tend + 1

    }
  }

  if (length(queue)) {
    if (if_free_servers()) {
      for (i in sample(1:m)) {
        if(servers$prog_num[i] == 0){
          servers$start[i] <- tsys
          servers$end[i] <- tsys + rexp(1, 1 / t2)
          servers$prog_num[i] <- queue[1]
          queue <- queue[-1]
          break
        }
      }
    }
  }

  tsys <- min(progs$end[1:k], servers$end[1:m])
}


In [ ]:
Tget_vec <- Tget_vec[Tget_vec$ready_flag == 1,]
paste <- "--------------------------------------------------------------"
View(paste)
View("среднее время до получения пользователем результатов реализации")
mean(Tget_vec$time)

Loch_vec_P <- Loch_vec / sum(Loch_vec)
Loch <- 0
for (i in 1:length(Loch_vec)) {
  Loch <- Loch + (i - 1) * Loch_vec_P[i]
}

View(paste)
View("среднее количество программ, ожидающих выполнения на сервере.")
Loch

Pn <- Pn_vec[2]/sum(Pn_vec)

View(paste)
View('вероятность того, что программа не будет выполнена сразу же, как только она поступила на терминал')
Pn

[1] "--------------------------------------------------------------"

[1] "среднее время до получения пользователем результатов реализации"

[1] 4.229722

[1] "--------------------------------------------------------------"

[1] "среднее количество программ, ожидающих выполнения на сервере."

[1] 0.9983791

[1] "--------------------------------------------------------------"

[1] "вероятность того, что программа не будет выполнена сразу же, как только она поступила на терминал"

[1] 0.531092

Теоретическая часть:

Составим формулу Эрланга

$$P_{0} = (1 + \frac{\frac{k!}{(k - 1)!}}{1!}y + \frac{\frac{k!}{(k - 2)!}}{2!}y^{2} + \frac{\frac{k!}{(k - 3)!}}{3!}y^{3} + ... + \frac{\frac{k!}{(k - m)!}}{m!}y^{m} + \frac{\frac{k!}{(k - m - 1)!}}{m!m}y^{m + 1} + \frac{\frac{k!}{(k - m - 2)!}}{m!m^{2}}y^{m + 2} + ... + \frac{\frac{k!}{(k - (k - 1))!}}{m!m^{k - m - 1}}y^{k - 1} + \frac{\frac{k!}{(k - k)!}}{m!m^{k - m}}y^{k})^{-1}$$

Свернём формулу

$$P_{0} = (\sum_{i = 0}^{m} \frac{\frac{k!}{(k - i)!}}{i!}y^{i} + \sum_{j = m + 1}^{k} \frac{\frac{k!}{(k - j)!}}{m!m^{j - m}}y^{j})^{-1}$$

Длина очереди

$$L_{och} = \sum_{r = 1}^{k - m} rP_{m + r} = \sum_{r = 1}^{k - m} r\frac{\frac{k!}{(k - m - r)!}}{m!m^{r}}y^{m + r} P_{0}$$

Вероятность необработки заявки

$$P_{busy} = \sum_{r = 0}^{k - m} P_{m + r} = \sum_{r = 0}^{k - m} \frac{\frac{k!}{(k - m - r)!}}{m!m^{r}}y^{m + r} P_{0}$$

Среднее число заявок в системе

$$L_{sys} = \sum_{i = 0}^{m} iP_{i} + \sum_{r = 1}^{k - m} (r+m)P_{m + r} = \sum_{i = 0}^{m} i\frac{\frac{k!}{(k - i)!}}{i!}y^{i}P_{0} + \sum_{r = 1}^{k - m} r\frac{\frac{k!}{(k - m - r)!}}{m!m^{r}}y^{m + r} P_{0}$$

Средняя интенсивность суммарного входного потока:

$$Λ = (k - L_{sys}) * λ$$


Время нахождения заявок в системе

$$T_{sys} = \frac{L_{sys}}{\Lambda} $$


Вычислим:

In [ ]:
lambda <- 1 / t1
mu <- 1 / t2
y <- lambda / mu
P0 <- 0

for(i in 0:m) {
  sum <-(factorial(k)/factorial(k - i))/factorial(i) * y^i
  P0 <- P0 + sum
}

for(j in (m + 1):k) {
  sum <-(factorial(k)/factorial(k - j)) / (factorial(m) * m^(j - m)) * y^j
  P0 <- P0 + sum
}

P0 <- 1 / P0
P0

[1] 0.05010991

Среднее количество программ, ожидающих выполнения на сервере:

In [ ]:
Loch <- 0

for (r in 1:(k - m)) {
  sum = (factorial(k)/factorial(k-m-r))/(factorial(m)*m^r)*y^(m+r)

  Loch <- Loch + r*sum*P0
}

Loch

[1] 0.9687048

Вероятность того, что программа не будет выполнена сразу же, как только она поступила на терминал:

In [ ]:
Pbusy <- 0

for (r in 0:(k - m)) {
  sum = (factorial(k)/factorial(k-m-r))/(factorial(m)*m^r)*y^(m+r)

  Pbusy <- Pbusy + sum*P0
}

Pbusy

[1] 0.5890987

In [ ]:
Lsys <- 0

for(i in 0:m) {
  sum <-(factorial(k)/factorial(k - i))/factorial(i) * y^i
  Lsys <- Lsys + i*sum*P0
}

for (r in 1:(k - m)) {
  sum = (factorial(k)/factorial(k-m-r))/(factorial(m)*m^r)*y^(m+r)

  Lsys <- Lsys + (r+m)*sum*P0
}

Lsys

[1] 3.307254

In [ ]:
ML = (k - Lsys) * lambda
ML

[1] 0.7795164

Среднее время до получения пользователем результатов реализации:

In [ ]:
Tsys = Lsys/ML
Tsys

[1] 4.2427

In [ ]:
paste <- "--------------------------------------------------------------"
View(paste)
View("среднее время до получения пользователем результатов реализации")
View(Tsys)
View(paste)
View("среднее количество программ, ожидающих выполнения на сервере")
View(Loch)
View(paste)
View('вероятность того, что программа не будет выполнена сразу же, как только она поступила на терминал')
View(Pbusy)

[1] "--------------------------------------------------------------"

[1] "среднее время до получения пользователем результатов реализации"

[1] 4.2427

[1] "--------------------------------------------------------------"

[1] "среднее количество программ, ожидающих выполнения на сервере."

[1] 0.9687048

[1] "--------------------------------------------------------------"

[1] "вероятность того, что программа не будет выполнена сразу же, как только она поступила на терминал"

[1] 0.5890987

Для ***m = 1***

---


Экспериментальная часть:

In [ ]:
m=1

Tend <- 10000

tsys <- 0

progs <- data.frame()
for (i in c(1:k)) {
  progs <- rbind(progs, c(0, 0, 0))
}
colnames(progs) <- c("end", "start", "deploy_num")

for (i in 1:k) {
  progs$end[i] <- tsys + rexp(1, 1 / t1)
  progs$start[i] <- 0
  progs$deploy_num <- 0
}



servers <- data.frame()
for (i in c(1:m)) {
  servers <- rbind(servers, c(0, 0, 0))
}
colnames(servers) <- c("end", "start", "prog_num")

for (i in 1:m) {
  servers$end[i] <- Tend + 1
  servers$start[i] <- 0
  servers$prog_num <- 0
}

counter_deploy <- 0
counter_ready <- 0


system_count <- 1
last_switch <- 0

Pn_vec <- c(0,0)
Loch_vec <- c(0)

Tget_vec <- data.frame()
Tget_vec <- rbind(Tget_vec, c(0,0))

colnames(Tget_vec) <- c("time", "ready_flag")

queue <- c()

ready_time <- Tend + 1

#функция проверяет, есть ли свободные сервера
if_free_servers <- function() {
  for (i in 1:m) {
    if (servers$prog_num[i] == 0) {
      return (1)
    }
  }
  return (0)
}


while (tsys < Tend) {

  plus1 <- length(queue) + 1
  if (is.na(Loch_vec[plus1])) {Loch_vec[plus1] <- 0}
  Loch_vec[plus1] <- Loch_vec[plus1] + tsys - last_switch
  last_switch <- tsys

  for (i in 1:k) {
    if (tsys == progs$end[i]){

      if (counter_deploy - counter_ready < m) { #если есть свободные сервера
        Pn_vec[1] <- Pn_vec[1] + 1
      }
      else {Pn_vec[2] <- Pn_vec[2] + 1} #если сервера заняты

      counter_deploy <- counter_deploy + 1
      progs$deploy_num[i] <- counter_deploy
      Tget_vec <- rbind(Tget_vec, c(0,0))
      Tget_vec$time[counter_deploy] <- progs$end[i]
      Tget_vec$ready_flag[counter_deploy] <- 0

      progs$end[i] <- Tend + 1

      queue <- append(queue, counter_deploy)
    }
  }


  for (i in 1:m) {
    if (tsys == servers$end[i]) {
      counter_ready <- counter_ready + 1
      Tget_vec$time[counter_ready] <- tsys - Tget_vec$time[counter_ready]
      Tget_vec$ready_flag[counter_ready] <- 1

      for (j in 1:k) {
        if (progs$deploy_num[j] == servers$prog_num[i]) {
          progs$start[j] = tsys
          progs$end[j] = tsys + rexp(1, 1 / t1)
        }
      }

      servers$prog_num[i] <- 0
      servers$end[i] <- Tend + 1

    }
  }

  if (length(queue)) {
    if (if_free_servers()) {
      for (i in sample(1:m)) {
        if(servers$prog_num[i] == 0){
          servers$start[i] <- tsys
          servers$end[i] <- tsys + rexp(1, 1 / t2)
          servers$prog_num[i] <- queue[1]
          queue <- queue[-1]
          break
        }
      }
    }
  }

  tsys <- min(progs$end[1:k], servers$end[1:m])
}

In [ ]:
Tget_vec <- Tget_vec[Tget_vec$ready_flag == 1,]
paste <- "--------------------------------------------------------------"
View(paste)
View("среднее время до получения пользователем результатов реализации")
mean(Tget_vec$time)

Loch_vec_P <- Loch_vec / sum(Loch_vec)
Loch <- 0
for (i in 1:length(Loch_vec)) {
  Loch <- Loch + (i - 1) * Loch_vec_P[i]
}

View(paste)
View("среднее количество программ, ожидающих выполнения на сервере.")
Loch

Pn <- Pn_vec[2]/sum(Pn_vec)

View(paste)
View('вероятность того, что программа не будет выполнена сразу же, как только она поступила на терминал')
Pn

[1] "--------------------------------------------------------------"

[1] "среднее время до получения пользователем результатов реализации"

[1] 28.78034

[1] "--------------------------------------------------------------"

[1] "среднее количество программ, ожидающих выполнения на сервере."

[1] 8.822817

[1] "--------------------------------------------------------------"

[1] "вероятность того, что программа не будет выполнена сразу же, как только она поступила на терминал"

[1] 0.9979526

Теоретическая часть:

Составим формулу Эрланга

$$P_{0} = (1 + \frac{\frac{k!}{(k - 1)!}}{1!}y + \frac{\frac{k!}{(k - 2)!}}{2!}y^{2} + \frac{\frac{k!}{(k - 3)!}}{3!}y^{3} + ... + \frac{\frac{k!}{(k - m)!}}{m!}y^{m} + \frac{\frac{k!}{(k - m - 1)!}}{m!m}y^{m + 1} + \frac{\frac{k!}{(k - m - 2)!}}{m!m^{2}}y^{m + 2} + ... + \frac{\frac{k!}{(k - (k - 1))!}}{m!m^{k - m - 1}}y^{k - 1} + \frac{\frac{k!}{(k - k)!}}{m!m^{k - m}}y^{k})^{-1}$$

Свернём формулу

$$P_{0} = (\sum_{i = 0}^{m} \frac{\frac{k!}{(k - i)!}}{i!}y^{i} + \sum_{j = m + 1}^{k} \frac{\frac{k!}{(k - j)!}}{m!m^{j - m}}y^{j})^{-1}$$

Длина очереди

$$L_{och} = \sum_{r = 1}^{k - m} rP_{m + r} = \sum_{r = 1}^{k - m} r\frac{\frac{k!}{(k - m - r)!}}{m!m^{r}}y^{m + r} P_{0}$$

Вероятность необработки заявки

$$P_{busy} = \sum_{r = 0}^{k - m} P_{m + r} = \sum_{r = 0}^{k - m} \frac{\frac{k!}{(k - m - r)!}}{m!m^{r}}y^{m + r} P_{0}$$

Среднее число заявок в системе

$$L_{sys} = \sum_{i = 0}^{m} iP_{i} + \sum_{r = 1}^{k - m} (r+m)P_{m + r} = \sum_{i = 0}^{m} i\frac{\frac{k!}{(k - i)!}}{i!}y^{i}P_{0} + \sum_{r = 1}^{k - m} r\frac{\frac{k!}{(k - m - r)!}}{m!m^{r}}y^{m + r} P_{0}$$

Средняя интенсивность суммарного входного потока:

$$Λ = (k - L_{sys}) * λ$$


Время нахождения заявок в системе

$$T_{sys} = \frac{L_{sys}}{\Lambda} $$


Вычислим:

In [ ]:
lambda <- 1 / t1
mu <- 1 / t2
y <- lambda / mu
P0 <- 0

for(i in 0:m) {
  sum <-(factorial(k)/factorial(k - i))/factorial(i) * y^i
  P0 <- P0 + sum
}

for(j in (m + 1):k) {
  sum <-(factorial(k)/factorial(k - j)) / (factorial(m) * m^(j - m)) * y^j
  P0 <- P0 + sum
}

P0 <- 1 / P0
P0

[1] 0.0001572563

Среднее количество программ, ожидающих выполнения на сервере:

In [ ]:
Loch <- 0

for (r in 1:(k - m)) {
  sum = (factorial(k)/factorial(k-m-r))/(factorial(m)*m^r)*y^(m+r)

  Loch <- Loch + r*sum*P0
}

Loch

[1] 9.000944

Вероятность того, что программа не будет выполнена сразу же, как только она поступила на терминал:

In [ ]:
Pbusy <- 0

for (r in 0:(k - m)) {
  sum = (factorial(k)/factorial(k-m-r))/(factorial(m)*m^r)*y^(m+r)

  Pbusy <- Pbusy + sum*P0
}

Pbusy

[1] 0.9998427

In [ ]:
Lsys <- 0

for(i in 0:m) {
  sum <-(factorial(k)/factorial(k - i))/factorial(i) * y^i
  Lsys <- Lsys + i*sum*P0
}

for (r in 1:(k - m)) {
  sum = (factorial(k)/factorial(k-m-r))/(factorial(m)*m^r)*y^(m+r)

  Lsys <- Lsys + (r+m)*sum*P0
}

Lsys

[1] 10.00079

In [ ]:
ML = (k - Lsys) * lambda
ML

[1] 0.3332809

Среднее время до получения пользователем результатов реализации:

In [ ]:
Tsys = Lsys/ML
Tsys

[1] 30.00708

In [ ]:
paste <- "--------------------------------------------------------------"
View(paste)
View("среднее время до получения пользователем результатов реализации")
View(Tsys)
View(paste)
View("среднее количество программ, ожидающих выполнения на сервере")
View(Loch)
View(paste)
View('вероятность того, что программа не будет выполнена сразу же, как только она поступила на терминал')
View(Pbusy)

[1] "--------------------------------------------------------------"

[1] "среднее время до получения пользователем результатов реализации"

[1] 30.00708

[1] "--------------------------------------------------------------"

[1] "среднее количество программ, ожидающих выполнения на сервере."

[1] 9.000944

[1] "--------------------------------------------------------------"

[1] "вероятность того, что программа не будет выполнена сразу же, как только она поступила на терминал"

[1] 0.9998427